In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import logging
import os
from pprint import pformat

import numpy as np
import pandas as pd
from hydra import compose, initialize
import matplotlib.pyplot as plt
from omegaconf import OmegaConf

from physped.omegaconf_resolvers import register_new_resolvers
from physped.io.readers import trajectory_reader
from physped.io.writers import save_piecewise_potential
from physped.preprocessing.trajectories import preprocess_trajectories
from physped.core.functions_to_discretize_grid import learn_potential_from_trajectories
from physped.core.trajectory_simulator import simulate_trajectories
from physped.core.functions_to_select_grid_piece import evaluate_selection_point, evaluate_selection_range, get_index_of_the_enclosing_bin
from physped.visualization.plot_trajectories import plot_trajectories
from physped.visualization.plot_discrete_grid import plot_discrete_grid
from physped.visualization.plot_histograms import create_all_histograms, plot_multiple_histograms
from physped.visualization.plot_1d_gaussian_fits import learn_piece_of_potential_plot

In [ ]:
# Load configuration parameters
env_name = "user_input"

with initialize(version_base=None, config_path="../conf", job_name="test_app"):
    config = compose(
        config_name="config",
        return_hydra_config=True,
        overrides=[
            f"params={env_name}", 
            # "params.data_source=local",
            ],
    )
    register_new_resolvers()

# set log level
log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

# set plot style
plt.style.use(Path.cwd().parent / "conf/science.mplstyle")

# change working directory
working_dir = config.hydra.run.dir
os.makedirs(working_dir, exist_ok=True)
os.chdir(working_dir)

In [ ]:
logging.info('CONFIGURATION FILE:\n' + pformat(OmegaConf.to_container(config), depth=1))

In [ ]:
logging.info('PARAMETERS:\n' + pformat(OmegaConf.to_container(config.params), depth=1))

In [ ]:
!wget https://zenodo.org/record/7007358/files/glow19_pnasnexus.csv

In [ ]:
## Read your own data

## E.g.
trajectories = pd.read_csv('glow19_pnasnexus.csv').iloc[:-2]
trajectories['time'] = pd.to_datetime(trajectories['time'])
# trajectories = trajectories[['time', 'tracked_object', 'x', 'y']].copy()
# trajectories.head(3)
logging.info('\n' + pformat(trajectories.head()))

In [ ]:
# Update parameters
config.params.fps = 30
xcol = config.params.colnames.xf = 'x'
ycol = config.params.colnames.yf = 'y'
config.params.colnames.Pid = 'tracked_object'
config.params.colnames.time = 'time'

# Infer the edge of the measurement domain from the data
xmin = config.params.grid.x.min = int(np.floor(trajectories[xcol].min()))
xmax = config.params.grid.x.max = int(np.ceil(trajectories[xcol].max()))
ymin = config.params.grid.y.min = int(np.floor(trajectories[ycol].min()))
ymax = config.params.grid.y.max = int(np.ceil(trajectories[ycol].max()))
config.params.default_xlims = [xmin, xmax]
config.params.default_ylims = [ymin, ymax]

logging.info('COLUMN NAMES:\n' + pformat(OmegaConf.to_container(config.params.colnames), depth=1))

In [ ]:
preprocessed_trajectories = preprocess_trajectories(trajectories, config=config)

In [ ]:
plot_trajectories(preprocessed_trajectories, config, "recorded")

In [ ]:
piecewise_potential = learn_potential_from_trajectories(preprocessed_trajectories, config)

In [ ]:
config.params.input_ntrajs = len(preprocessed_trajectories.Pid.unique())
simulated_trajectories = simulate_trajectories(piecewise_potential, config)

In [ ]:
plot_trajectories(preprocessed_trajectories, config, "simulated")

In [ ]:
observables = ["xf", "yf", "uf", "vf"]
config.params.simulation.ntrajs = len(simulated_trajectories.Pid.unique())
histograms = create_all_histograms(preprocessed_trajectories, simulated_trajectories, config)
plot_multiple_histograms(observables, histograms, "PDF", config)